In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Apr_dict_JSON.json', 'r') as json_file:
    link_day_minute_Apr_dict_JSON = json.load(json_file)

In [10]:
weekend_Apr_list = [1, 7, 8, 14, 15, 21, 22, 28, 29]

In [11]:
link_day_minute_Apr_list = []
for link_idx in range(24):
    for day in weekend_Apr_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Apr_list.append(link_day_minute_Apr_dict_JSON[key] ['AM_flow_minute'][minute_idx])

In [12]:
len(link_day_minute_Apr_list)

25920

In [13]:
x = np.matrix(link_day_minute_Apr_list)
x = np.matrix.reshape(x, 24, 1080)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [14]:
x[:,:2]

matrix([[ 348542.80813655,  345259.62645033],
        [ 355086.7459287 ,  362326.11630033],
        [ 629648.81511729,  643692.80417731],
        [ 670875.04781719,  672386.57959279],
        [ 168161.41518738,  166445.30171351],
        [ 224655.56228273,  230507.86537481],
        [ 414886.06217914,  418039.39635458],
        [ 407349.94018725,  400727.96495742],
        [ 375670.11875303,  371801.63838264],
        [ 398099.30195893,  390149.12125805],
        [ 373611.81792515,  364204.22027516],
        [ 343596.84118445,  331173.77581141],
        [  91858.09207355,   90596.04918198],
        [ 102720.59466785,   95587.77120316],
        [ 154962.17220949,  160158.6148157 ],
        [ 134942.07398453,  139383.66470396],
        [ 210545.0613289 ,  212496.9720784 ],
        [ 224343.48804241,  217649.99691815],
        [ 379221.9535908 ,  359926.6303642 ],
        [ 344032.19496048,  334708.70304444],
        [ 505519.19006592,  500909.55544178],
        [ 499398.62081751,  539719

In [15]:
np.size(x, 0), np.size(x, 1)

(24, 1080)

In [16]:
np.size(A,0), np.size(A,1)

(24, 140)

In [17]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [7e-02, 9e+00]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 24
 AA' NZ     : 2.760e+02
 Factor NZ  : 3.000e+02
 Factor Ops : 4.900e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -8.81827548e+06  0.00000000e+00  5.27e+03 9.89e+02  1.00e+06     0s
   1  -4.60799733e+06 -2.95312641e+06  1.42e+03 2.67e+02  3.03e+05     0s
   2  -3.27131219e+06 -4.54046075e+06  1.86e+02 3.50e+01  6.45e+04     0s
   3  -3.47295689e+06 -3.92261173e+06  1.86e-04 3.50e-05  8.0

In [18]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Apr_weekend.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [19]:
lam_list

[8.413870140088149e-10,
 86843.3126083412,
 142993.480437929,
 63986.2990702229,
 22598.26583716808,
 1.1185272503166471e-07,
 1.1234978796461302e-07,
 141827.65812730882,
 5.2043383101937554e-11,
 137067.75177975596,
 5.204338157074352e-11,
 5.204337473699751e-11,
 5.20192221252605e-11,
 5.2369879901062766e-11,
 1.306648845735391e-07,
 1.3040549505151123e-07,
 2.2770338665155273e-08,
 199765.09880834597,
 41255.284408466076,
 9.695050925267334e-08,
 1.1172455710248014e-07,
 1.5418947157634774e-06,
 224557.6522977542,
 4.881095727883285e-06,
 42654.344006857464,
 13658.00712608882,
 6.567947179078639e-08,
 64118.797917694086,
 4.167798622101843e-06,
 4.192762435798608e-06,
 1.3694790014042758e-06,
 6.617575789531825e-07,
 26504.89742692344,
 265885.0151996791,
 6.574882460141048e-08,
 37951.797395919464,
 8563.969570813084,
 27974.66245272591,
 47670.04974698958,
 320.7350072617871,
 512366.22035066347,
 24096.241446412525,
 360187.2763634367,
 29517.392497688434,
 9.415470104810362e-0